In [1]:
from glob import glob
import pandas as pd
from scipy.stats import mannwhitneyu
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skbio.stats.composition import clr
from matplotlib_venn import venn3

sns.set_context('talk')
sns.set(rc={"figure.dpi":150, 'savefig.dpi':300})
sns.set_style('whitegrid')

def p_adjust(pvalues, method='fdr_bh'):
    res = multipletests(pvalues, method=method)
    return np.array(res[1], dtype=float)

# Metabolomics associations
##### 7/18/22
##### Michael Shaffer
##### Merck ESC, Sys bio group

After some success with correlating KOs with continuous titer we decided to use the same approach on the metabolomics data. This data table came from Tom directly.

## Read in data

Read in the excel table from Tom and fill out empty cells with zeros.

In [2]:
metab_data = pd.read_excel('../../data/metabolomics_abunds.xlsx', index_col=0).fillna(0)
metab_data = metab_data.drop('2-Methyl-1-butanol', axis=1) # drop 2-methyl-1-butanol, bad annotation
metab_data.head()

,GCDCA,GDCA,GHDCA or GUDCA,CA,TCDCA,TCA,CDCA,7oxoCA,TUDCA,UDCA or HDCA,...,Uracil,Adenosine,Phenaceturic acid,N-Acetyl D-galactosamine,Pyridoxal hydrochloride,2-Deoxyuridine,Vanillic acid,Thymine,Nicotinic acid,Homocitrate
Compound name,,,,,,,,,,,,,,,,,,,,,
P103_V12_05052020,13165.576990,6486.205847,1037.427698,877.477368,842.441112,428.399288,412.663454,6.309238,26.103778,37.617317,...,571,112,45,90,426,1223,6865,549,217,3044
P106_V9_04022019,5324.081005,615.267690,1431.812977,536.910350,477.807838,761.254736,55.773899,4.142229,91.815489,19.861706,...,400,368,206,217,201,372,53,230,216,109
P107_A1_04152019,11967.195030,3.048386,8876.579286,884.309058,2720.613568,3492.197205,106.185518,7.807234,664.956760,113.460800,...,594,112,8,330,102,9,87,55,177,6952
P108_V9_04022019,117.128627,-0.240781,116.125243,29.790592,6.916959,5.775257,1.574994,0.418815,1.671431,4.996911,...,60,226,11,16,249,104,64,18,24,36
P108_V12_05212020,16651.224940,1707.969274,3575.590198,1277.281488,2356.663104,1109.351448,204.781749,41.696046,216.707453,107.961759,...,375,184,205,141,253,702,298,196,196,3468


In [3]:
len(metab_data.columns)

111

111 total compounds.

In [4]:
(metab_data < 0).sum().sum()

34

There are only 34 negative values in the data set so not much to worry about.

In [5]:
metab_data.unstack()[metab_data.unstack() < 0]

        Compound name    
GDCA    P108_V9_04022019    -0.240781
        P134_V9_12062019    -3.462650
        P136_V9_01092020    -3.398780
        P136_V12_01042021   -0.059861
        P202_V12_02252020   -1.708202
        P212_V5_05232018    -2.539283
        P215_V9_03292019    -4.984036
        P215_V12_03302020   -2.424750
        P217_V5_06122018    -1.613075
        P217_V9_04122019    -0.517431
        P218_V5_06062018    -0.427326
        P220_V5_06132018    -0.218075
        P224_V9_05292019    -0.396865
        P229_V5_07022018    -0.664653
        P235_V5_08152018    -0.203266
        P237_V9_06242019    -0.407863
        P239_V5_08292018    -0.942639
        P246_V10_10032019   -1.541833
        P250_V5_11062018    -1.424605
        P250_V9_09092019    -2.380330
        P252_V9_09102019    -1.033160
        P256_V9_10082018    -0.539685
        P258_V5_01032019    -1.193836
        P260_V9_11072019    -7.410659
        P261_V5_01032019    -2.489533
        P263_V9_11262019

There are negative numbers in the data set. Checked with Tom and he said that these can be treated as zeros. So do that.

In [6]:
# Replace negative numbers with 0 as Tom said in an email
metab_data[metab_data < 0] = 0
(metab_data == 0).sum().sum()

38

Relative abundance normalized version of table is calculated by summing rows and dividing column values by totals. Makes it so that values are percent abundances of sample.

In [7]:
metab_data_rel = metab_data.div(metab_data.sum(axis=1), axis=0)
metab_data_rel.head()

,GCDCA,GDCA,GHDCA or GUDCA,CA,TCDCA,TCA,CDCA,7oxoCA,TUDCA,UDCA or HDCA,...,Uracil,Adenosine,Phenaceturic acid,N-Acetyl D-galactosamine,Pyridoxal hydrochloride,2-Deoxyuridine,Vanillic acid,Thymine,Nicotinic acid,Homocitrate
Compound name,,,,,,,,,,,,,,,,,,,,,
P103_V12_05052020,0.001680,8.275956e-04,0.000132,0.000112,0.000107,0.000055,5.265304e-05,8.050157e-07,3.330664e-06,0.000005,...,0.000073,0.000014,0.000006,0.000011,0.000054,0.000156,0.000876,0.000070,0.000028,0.000388
P106_V9_04022019,0.000847,9.783810e-05,0.000228,0.000085,0.000076,0.000121,8.869005e-06,6.586853e-07,1.460024e-05,0.000003,...,0.000064,0.000059,0.000033,0.000035,0.000032,0.000059,0.000008,0.000037,0.000034,0.000017
P107_A1_04152019,0.002120,5.401497e-07,0.001573,0.000157,0.000482,0.000619,1.881523e-05,1.383380e-06,1.178250e-04,0.000020,...,0.000105,0.000020,0.000001,0.000058,0.000018,0.000002,0.000015,0.000010,0.000031,0.001232
P108_V9_04022019,0.000065,0.000000e+00,0.000065,0.000017,0.000004,0.000003,8.751132e-07,2.327061e-07,9.286966e-07,0.000003,...,0.000033,0.000126,0.000006,0.000009,0.000138,0.000058,0.000036,0.000010,0.000013,0.000020
P108_V12_05212020,0.002614,2.681638e-04,0.000561,0.000201,0.000370,0.000174,3.215225e-05,6.546587e-06,3.402467e-05,0.000017,...,0.000059,0.000029,0.000032,0.000022,0.000040,0.000110,0.000047,0.000031,0.000031,0.000545


CLR normalized version of table is calculated using CLR (centered log ratio transformation) which controls for compositionality. This is done using the scikit-bio package.

In [8]:
metab_data_clr = pd.DataFrame(clr(metab_data + .001), index=metab_data.index, columns=metab_data.columns)

Build simple metadata table based on the sample names from the metabolomics table.

In [9]:
meta_rows = [[i, int(i.split('_')[0].strip('P')), i.split('_')[1], i.split('_')[-1]] for i in metab_data.index]
meta_base = pd.DataFrame(meta_rows, columns=['SampleID', 'BabyN', 'VisitCode', 'VisitDate']).set_index('SampleID')
meta_base.head()

,BabyN,VisitCode,VisitDate
SampleID,,,
P103_V12_05052020,103,V12,05052020
P106_V9_04022019,106,V9,04022019
P107_A1_04152019,107,A1,04152019
P108_V9_04022019,108,V9,04022019
P108_V12_05212020,108,V12,05212020


Connect sample data to 1 yr titer data via the baby numbers.

In [10]:
titer_data = pd.read_csv('../../data/vaccine_response/vaccine_response_y1.tsv', sep='\t', index_col=0)
titer_data.index = [int(i.split('Baby')[-1]) for i in titer_data.index]
titer_data.head()

,PT,Dip,FHA,PRN,TET,PRP (Hib),PCV ST1,PCV ST3,PCV ST4,PCV ST5,...,median_mmNorm,median_mmNorm_DTAPHib,median_mmNorm_PCV,PT_protected,Dip_protected,FHA_protected,PRN_protected,TET_protected,PRP (Hib)_protected,VR_group
106,2.5,0.21,11.0,2.5,0.30,0.39,141.0,35.0,56.0,139.0,...,0.061955,0.052874,0.061955,False,True,True,False,True,True,NVR
107,2.5,0.44,3.0,9.0,0.52,1.60,2430.0,415.0,194.0,332.0,...,0.449483,0.114018,0.958142,False,True,False,True,True,True,NVR
108,2.5,0.05,1.5,2.5,0.05,0.27,21.0,3.0,24.0,41.0,...,0.000000,0.000000,0.003102,False,False,False,False,False,True,LVR
109,27.0,NaN,NaN,63.0,1.35,7.02,NaN,NaN,NaN,NaN,...,0.700925,0.763049,0.486810,True,False,False,True,True,True,NVR
110,14.0,0.24,15.0,20.0,2.45,NaN,301.0,63.0,400.0,289.0,...,0.266219,0.284211,0.245121,True,True,True,True,True,False,NVR


Use BabyN to connect compound abundance to titer values since there is only one 2 month sample and one set of 1 year titer values per baby.

In [11]:
per_sample_titer_data = pd.DataFrame({sample: titer_data.loc[i] for sample, i in meta_base['BabyN'].iteritems() if i in titer_data.index}).transpose()
per_sample_titer_data.head()

,PT,Dip,FHA,PRN,TET,PRP (Hib),PCV ST1,PCV ST3,PCV ST4,PCV ST5,...,median_mmNorm,median_mmNorm_DTAPHib,median_mmNorm_PCV,PT_protected,Dip_protected,FHA_protected,PRN_protected,TET_protected,PRP (Hib)_protected,VR_group
P106_V9_04022019,2.5,0.21,11.0,2.5,0.3,0.39,141.0,35.0,56.0,139.0,...,0.061955,0.052874,0.061955,False,True,True,False,True,True,NVR
P107_A1_04152019,2.5,0.44,3.0,9.0,0.52,1.6,2430.0,415.0,194.0,332.0,...,0.449483,0.114018,0.958142,False,True,False,True,True,True,NVR
P108_V9_04022019,2.5,0.05,1.5,2.5,0.05,0.27,21.0,3.0,24.0,41.0,...,0.0,0.0,0.003102,False,False,False,False,False,True,LVR
P108_V12_05212020,2.5,0.05,1.5,2.5,0.05,0.27,21.0,3.0,24.0,41.0,...,0.0,0.0,0.003102,False,False,False,False,False,True,LVR
P110_V9_05172019,14.0,0.24,15.0,20.0,2.45,NaN,301.0,63.0,400.0,289.0,...,0.266219,0.284211,0.245121,True,True,True,True,True,False,NVR


Merge sample metadata and titer data, filter out samples without one year titers.

In [12]:
meta = pd.concat([meta_base, per_sample_titer_data], axis=1)
meta = meta.loc[~pd.isna(meta['VR_group'])]
meta.head()

,BabyN,VisitCode,VisitDate,PT,Dip,FHA,PRN,TET,PRP (Hib),PCV ST1,...,median_mmNorm,median_mmNorm_DTAPHib,median_mmNorm_PCV,PT_protected,Dip_protected,FHA_protected,PRN_protected,TET_protected,PRP (Hib)_protected,VR_group
P106_V9_04022019,106,V9,04022019,2.5,0.21,11.0,2.5,0.3,0.39,141.0,...,0.061955,0.052874,0.061955,False,True,True,False,True,True,NVR
P107_A1_04152019,107,A1,04152019,2.5,0.44,3.0,9.0,0.52,1.6,2430.0,...,0.449483,0.114018,0.958142,False,True,False,True,True,True,NVR
P108_V9_04022019,108,V9,04022019,2.5,0.05,1.5,2.5,0.05,0.27,21.0,...,0.0,0.0,0.003102,False,False,False,False,False,True,LVR
P108_V12_05212020,108,V12,05212020,2.5,0.05,1.5,2.5,0.05,0.27,21.0,...,0.0,0.0,0.003102,False,False,False,False,False,True,LVR
P110_V9_05172019,110,V9,05172019,14.0,0.24,15.0,20.0,2.45,NaN,301.0,...,0.266219,0.284211,0.245121,True,True,True,True,True,False,NVR


Find shared samples between the metadata and metabolomics data. Filter metadata to match the metabolomics.

In [13]:
in_both = list(set(meta.index) & set(metab_data.index))
meta_matched = meta.loc[in_both]
print(meta_matched.shape)

(141, 51)


In [14]:
meta_matched['VisitCode'].value_counts()

V9     52
V12    44
V5     36
V10     4
A1      2
S3      1
A2      1
A3      1
Name: VisitCode, dtype: int64

Plurality of samples from 1 year. Many from 2 months and 2 years as well. We will focus on two months (V5) and 1 year (V9).

## Do correlation with un-normalized data

First take the "raw" data (not further normalized from what Tom gave me) and correlate each feature with the mei.

### 2 months (V5) correlations

First focus on the 2 month time points.

We are using spearman for all of these correlations and multiple testing correction with Benjamini-Hochberg.

In [15]:
meta_v5 = meta_matched.query("VisitCode == 'V5'")
metab_data_v5 = metab_data.loc[meta_v5.index]
metab_stats_v5_rows = list()
for metabolite, row in metab_data_v5.transpose().iterrows():
    lvr_abunds = row[meta_v5.query('VR_group == "LVR"').index]
    nvr_abunds = row[meta_v5.query('VR_group == "NVR"').index]
    # check for not all zeros
    # lvr_gt_20 = (lvr_abunds != 0).sum()/len(lvr_abunds) > .2
    # nvr_gt_20 = (nvr_abunds != 0).sum()/len(nvr_abunds) > .2
    lvr_gt_20 = lvr_abunds.sum()/len(lvr_abunds) > 10
    nvr_gt_20 = nvr_abunds.sum()/len(nvr_abunds) > 10
    if lvr_gt_20 or nvr_gt_20:
        stat, p_value = mannwhitneyu(lvr_abunds, nvr_abunds)
        metab_stats_v5_rows.append([metabolite, lvr_abunds.mean(), nvr_abunds.mean(), stat, p_value])
metab_stats_v5 = pd.DataFrame(metab_stats_v5_rows, columns=['metabolite', 'LVR_mean', 'NVR_mean', 'statistic', 'p_value']).sort_values('p_value')
metab_stats_v5['p_adj'] = p_adjust(metab_stats_v5['p_value'])
metab_stats_v5.head()

,metabolite,LVR_mean,NVR_mean,statistic,p_value,p_adj
13,serotonin,21332.737790,10654.264478,109.0,0.020677,1.0
12,5-HIAA,4568.590914,2555.333651,106.0,0.033036,1.0
81,Phenylpyruvic acid,4399.250000,2038.093750,102.0,0.057143,1.0
53,4-hydroxyphenylacetate,6244.250000,2738.843750,100.0,0.073926,1.0
40,Maleic acid,13727.750000,9218.843750,98.0,0.092488,1.0


### 1 year (V9) correlations

Same correlations but with year 1 un-normalized data.

In [16]:
meta_v9 = meta_matched.query("VisitCode == 'V9'")
metab_data_v9 = metab_data.loc[meta_v9.index]
metab_stats_v9_rows = list()
for metabolite, row in metab_data_v9.transpose().iterrows():
    lvr_abunds = row[meta_v9.query('VR_group == "LVR"').index]
    nvr_abunds = row[meta_v9.query('VR_group == "NVR"').index]
    # check for not all zeros
    # lvr_gt_20 = (lvr_abunds != 0).sum()/len(lvr_abunds) > .2
    # nvr_gt_20 = (nvr_abunds != 0).sum()/len(nvr_abunds) > .2
    lvr_gt_20 = lvr_abunds.sum()/len(lvr_abunds) > 10
    nvr_gt_20 = nvr_abunds.sum()/len(nvr_abunds) > 10
    if lvr_gt_20 or nvr_gt_20:
        stat, p_value = mannwhitneyu(lvr_abunds, nvr_abunds)
        metab_stats_v9_rows.append([metabolite, lvr_abunds.mean(), nvr_abunds.mean(), stat, p_value])
metab_stats_v9 = pd.DataFrame(metab_stats_v9_rows, columns=['metabolite', 'LVR_mean', 'NVR_mean', 'statistic', 'p_value']).sort_values('p_value')
metab_stats_v9['p_adj'] = p_adjust(metab_stats_v9['p_value'])
metab_stats_v9 = metab_stats_v9.set_index('metabolite')
metab_stats_v9.query('p_adj < .05')

,LVR_mean,NVR_mean,statistic,p_value,p_adj
metabolite,,,,,
L-Citrulline,2723.800000,6113.428571,22.0,0.000013,0.000827
L-Serine,4132.800000,11121.142857,23.0,0.000015,0.000827
L-Histidine,433.700000,955.476190,40.0,0.000083,0.002456
L-asparagine,839.700000,1450.380952,43.5,0.000116,0.002456
L-Methionine,8814.600000,18713.357143,45.0,0.000134,0.002456
L-Threonine,18041.100000,40270.000000,46.0,0.000147,0.002456
L-Proline,3105.500000,7522.000000,48.0,0.000177,0.002456
L-Tyrosine,21412.700000,41665.071429,48.0,0.000177,0.002456
L-Glutamine,8473.300000,16422.500000,50.0,0.000213,0.002625


In [17]:
len(metab_stats_v9.query('p_adj < .05'))

49

49 compounds have significant adjusted p-values. o about half of things are significant. Is this an artifact or does it mean that base metabolism is associated with titer response?

Tryptophan D5 is in this list. It is a control compound so we probably shouldn't trust this.

In [18]:
metab_stats_v9.loc[[i for i in metab_stats_v9.index if 'D5' in i]]

,LVR_mean,NVR_mean,statistic,p_value,p_adj
metabolite,,,,,
L-tryptophan-D5,26618.0,18953.452381,332.0,0.004787,0.016606
